In [1]:
#%
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tqdm
import cx_Oracle


In [5]:
cx_Oracle.init_oracle_client(lib_dir=r"C:/Oracle/instantclient_19_19")

In [53]:
#%
connection = cx_Oracle.connect(user='scott', password='tiger', dsn='192.168.119.119:1521/dink11.dbsvr')
cursor = connection.cursor()
sql="select categorySmall, convenienceToEmbedding from CATEGORYembedding"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['categorySmall','convenienceToEmbedding']
category = pd.DataFrame(rows, columns=columns)
# category



In [54]:
sql="SELECT c.email as email,ch.categorySmall, ch.amount FROM cardhistory ch JOIN card c ON ch.cardid = c.cardid"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['tmpEmail','categorySmall','amount']
data = pd.DataFrame(rows, columns=columns)
data

,tmpEmail,categorySmall,amount
0,gangjimin@live.com,일식/중식/양식,46000
1,gangjimin@live.com,제과점,2500
2,gangjimin@live.com,편의점,22700
3,gangjimin@live.com,편의점,16300
4,gangjimin@live.com,제과점,10500
...,...,...,...
199995,gimgyeongsu@nate.com,한식,25000
199996,gimgyeongsu@nate.com,미용실,80000
199997,gimgyeongsu@nate.com,제과점,3800
199998,gimgyeongsu@nate.com,주유소,50000


In [55]:
data['categoryEmbedding'] = data['categorySmall'].replace(category.set_index('categorySmall')['convenienceToEmbedding'])
data

,tmpEmail,categorySmall,amount,categoryEmbedding
0,gangjimin@live.com,일식/중식/양식,46000,0.212529
1,gangjimin@live.com,제과점,2500,0.432291
2,gangjimin@live.com,편의점,22700,1.000000
3,gangjimin@live.com,편의점,16300,1.000000
4,gangjimin@live.com,제과점,10500,0.432291
...,...,...,...,...
199995,gimgyeongsu@nate.com,한식,25000,0.178922
199996,gimgyeongsu@nate.com,미용실,80000,0.399781
199997,gimgyeongsu@nate.com,제과점,3800,0.432291
199998,gimgyeongsu@nate.com,주유소,50000,0.369205


In [56]:
userList=data['tmpEmail'].unique()

In [57]:
data.head()

,tmpEmail,categorySmall,amount,categoryEmbedding
0,gangjimin@live.com,일식/중식/양식,46000,0.212529
1,gangjimin@live.com,제과점,2500,0.432291
2,gangjimin@live.com,편의점,22700,1.000000
3,gangjimin@live.com,편의점,16300,1.000000
4,gangjimin@live.com,제과점,10500,0.432291


In [81]:
mode_card_df = {'tmpEmail': [], 'categorySmall' : [], 'embedding_mode' : [],'amount':[]}
for i, userId in tqdm.tqdm(enumerate(userList)):
    '''
    _df : 카드 거래 내역 중 카드번호 같은 것 끼리 모은 임시 df
    mean_card_df : 카드번호 같은 것들 중, 최빈값 업종을 뽑아서 대표 업종으로 만든 df
    '''
    _df = data[data['tmpEmail']==userId][['categorySmall', 'categoryEmbedding','amount']]
    # print(_df)
    # print(_df['categorySmall'].mode().iloc[0])
    # print(_df['amount'].mean())

    mode_card_df['tmpEmail'].append(userId)
    mode_card_df['categorySmall'].append(_df['categorySmall'].mode().iloc[0])
    mode_card_df['embedding_mode'].append(_df['categoryEmbedding'].mode().iloc[0])
    mode_card_df['amount'].append(_df['amount'].mean())
    
# mode_card_df = pd.DataFrame(mode_card_df)
# print(mode_card_df)
print(len(mode_card_df['tmpEmail']))
print(len(mode_card_df['categorySmall']))
print(len(mode_card_df['amount']))
mode_card_df = pd.DataFrame(mode_card_df)
mode_card_df['amount'] = mode_card_df['amount'].astype(int)
mode_card_df

0it [00:00, ?it/s]

200it [00:03, 62.36it/s]

200
200
200


,tmpEmail,categorySmall,embedding_mode,amount
0,gangjimin@live.com,한식,0.178922,62578
1,coeminji@dreamwiz.com,한식,0.178922,64833
2,baedohyeon@hotmail.com,편의점,1.000000,72944
3,hyejinbag@hanmail.net,한식,0.178922,61646
4,igim@daum.net,한식,0.178922,68379
...,...,...,...,...
195,gimyeongceol@nate.com,일식/중식/양식,0.212529,63328
196,eunseo08@hanmail.net,한식,0.178922,73743
197,gimgyeongsu@nate.com,한식,0.178922,92952
198,gyeongja17@hanmail.net,한식,0.178922,71376


In [85]:
# mode_card_df['embedding_mode'].values
# mode_card_df['amount'].values

In [86]:
from sklearn.cluster import KMeans

n_clusters = 4
X = np.array(list(zip(mode_card_df['embedding_mode'].values, mode_card_df['amount'].values)))
print(X.shape)

kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
cluster_labels = kmeans.labels_
print(len(cluster_labels))
mode_card_df['Cluster'] = cluster_labels

mode_card_df

(200, 2)
200


,tmpEmail,categorySmall,embedding_mode,amount,Cluster
0,gangjimin@live.com,한식,0.178922,62578,3
1,coeminji@dreamwiz.com,한식,0.178922,64833,3
2,baedohyeon@hotmail.com,편의점,1.000000,72944,3
3,hyejinbag@hanmail.net,한식,0.178922,61646,1
4,igim@daum.net,한식,0.178922,68379,3
...,...,...,...,...,...
195,gimyeongceol@nate.com,일식/중식/양식,0.212529,63328,3
196,eunseo08@hanmail.net,한식,0.178922,73743,3
197,gimgyeongsu@nate.com,한식,0.178922,92952,2
198,gyeongja17@hanmail.net,한식,0.178922,71376,3


In [87]:
mode_card_df['Cluster'].value_counts()

3    86
2    40
0    38
1    36
Name: Cluster, dtype: int64